### Class 7- Starter code


In [1]:
import numpy as np
import pandas as pd
from sklearn import linear_model, metrics


### Create sample data and fit a model

In [2]:
df = pd.DataFrame({'x': range(100), 'y': range(100)})
biased_df  = df.copy()
biased_df.loc[:20, 'x'] = 1
biased_df.loc[:20, 'y'] = 1

def append_jitter(series):
    jitter = np.random.random_sample(size=100)
    return series + jitter

df['x'] = append_jitter(df.x)
df['y'] = append_jitter(df.y)

biased_df['x'] = append_jitter(biased_df.x)
biased_df['y'] = append_jitter(biased_df.y)


In [5]:
biased_df.head(20)

,x,y
0,1.129258,1.464573
1,1.272979,1.574740
2,1.520057,1.608695
3,1.502146,1.156629
4,1.630134,1.244508
5,1.508180,1.306053
6,1.404405,1.681649
7,1.174945,1.234936
8,1.898274,1.553831
9,1.796497,1.023354


In [6]:
## fit
lm = linear_model.LinearRegression().fit(df[['x']], df['y'])
print metrics.mean_squared_error(df['y'], lm.predict(df[['x']]))


0.165691843884


In [7]:
## biased fit
lm = linear_model.LinearRegression().fit(biased_df[['x']], biased_df['y'])
print metrics.mean_squared_error(df['y'], lm.predict(df[['x']]))

0.172038962929


## Cross validation
#### Intro to cross validation with bike share data from last time. We will be modeling casual ridership. 

In [15]:
from sklearn import cross_validation
wd = '../../assets/dataset/'
bikeshare = pd.read_csv(wd + 'bikeshare.csv')

#### Create dummy variables and set outcome (dependent) variable

In [16]:
weather = pd.get_dummies(bikeshare.weathersit, prefix='weather')
modeldata = bikeshare[['temp', 'hum']].join(weather[['weather_1', 'weather_2', 'weather_3']])
y = bikeshare.casual 

#### Create a cross valiation with 5 folds

In [29]:
for k in range(2,51,2):
    kf = cross_validation.KFold(len(modeldata), n_folds=k, shuffle=True)
    mse_values = []
    scores = []
    for train_index, test_index in kf:
        lm = linear_model.LinearRegression().fit(modeldata.iloc[train_index], y.iloc[train_index])
        mse_values.append(metrics.mean_squared_error(y.iloc[test_index], lm.predict(modeldata.iloc[test_index])))
        scores.append(lm.score(modeldata, y))
    print "~~~~ SUMMARY OF CROSS VALIDATION {} FOLDS~~~~".format(k)
    print 'Mean of MSE for all folds:', np.mean(mse_values)
    print 'Mean of R2 for all folds:', np.mean(scores)
    print

~~~~ SUMMARY OF CROSS VALIDATION 2 FOLDS~~~~
Mean of MSE for all folds: 1679.25617107
Mean of R2 for all folds: 0.311018432523

~~~~ SUMMARY OF CROSS VALIDATION 4 FOLDS~~~~
Mean of MSE for all folds: 1673.81122145
Mean of R2 for all folds: 0.311858595618

~~~~ SUMMARY OF CROSS VALIDATION 6 FOLDS~~~~
Mean of MSE for all folds: 1673.54198926
Mean of R2 for all folds: 0.311899152085

~~~~ SUMMARY OF CROSS VALIDATION 8 FOLDS~~~~
Mean of MSE for all folds: 1673.26181318
Mean of R2 for all folds: 0.311915798806

~~~~ SUMMARY OF CROSS VALIDATION 10 FOLDS~~~~
Mean of MSE for all folds: 1674.1875474
Mean of R2 for all folds: 0.311899834763

~~~~ SUMMARY OF CROSS VALIDATION 12 FOLDS~~~~
Mean of MSE for all folds: 1673.37170846
Mean of R2 for all folds: 0.311920474081

~~~~ SUMMARY OF CROSS VALIDATION 14 FOLDS~~~~
Mean of MSE for all folds: 1673.30146058
Mean of R2 for all folds: 0.311923957325

~~~~ SUMMARY OF CROSS VALIDATION 16 FOLDS~~~~
Mean of MSE for all folds: 1673.24256866
Mean of R2 for 

In [18]:
kf = cross_validation.KFold(len(modeldata), n_folds=5, shuffle=True)

In [19]:
mse_values = []
scores = []
n= 0
print "~~~~ CROSS VALIDATION each fold ~~~~"
for train_index, test_index in kf:
    lm = linear_model.LinearRegression().fit(modeldata.iloc[train_index], y.iloc[train_index])
    mse_values.append(metrics.mean_squared_error(y.iloc[test_index], lm.predict(modeldata.iloc[test_index])))
    scores.append(lm.score(modeldata, y))
    n+=1
    print 'Model', n
    print 'MSE:', mse_values[n-1]
    print 'R2:', scores[n-1]


print "~~~~ SUMMARY OF CROSS VALIDATION ~~~~"
print 'Mean of MSE for all folds:', np.mean(mse_values)
print 'Mean of R2 for all folds:', np.mean(scores)

~~~~ CROSS VALIDATION each fold ~~~~
Model 1
MSE: 1666.71067131
R2: 0.311886829928
Model 2
MSE: 1645.08529223
R2: 0.311884879068
Model 3
MSE: 1719.99104889
R2: 0.311904770979
Model 4
MSE: 1733.24404552
R2: 0.311860590413
Model 5
MSE: 1604.20242517
R2: 0.311846209538
~~~~ SUMMARY OF CROSS VALIDATION ~~~~
Mean of MSE for all folds: 1673.84669662
Mean of R2 for all folds: 0.311876655985


In [21]:
lm = linear_model.LinearRegression().fit(modeldata, y)
print "~~~~ Single Model ~~~~"
print 'MSE of single model:', metrics.mean_squared_error(y, lm.predict(modeldata))
print 'R2: ', lm.score(modeldata, y)

~~~~ Single Model ~~~~
MSE of single model: 1672.58110765
R2:  0.311934605989


### Check
While the cross validated approach here generated more overall error, which of the two approaches would predict new data more accurately: the single model or the cross validated, averaged one? Why?


Answer: 

### There are ways to improve our model with regularization. 
Let's check out the effects on MSE and R2

In [22]:
lm = linear_model.LinearRegression().fit(modeldata, y)
print "~~~ OLS ~~~"
print 'OLS MSE: ', metrics.mean_squared_error(y, lm.predict(modeldata))
print 'OLS R2:', lm.score(modeldata, y)

lm = linear_model.Lasso().fit(modeldata, y)
print "~~~ Lasso ~~~"
print 'Lasso MSE: ', metrics.mean_squared_error(y, lm.predict(modeldata))
print 'Lasso R2:', lm.score(modeldata, y)

lm = linear_model.Ridge().fit(modeldata, y)
print "~~~ Ridge ~~~"
print 'Ridge MSE: ', metrics.mean_squared_error(y, lm.predict(modeldata))
print 'Ridge R2:', lm.score(modeldata, y)

~~~ OLS ~~~
OLS MSE:  1672.58110765
OLS R2: 0.311934605989
~~~ Lasso ~~~
Lasso MSE:  1725.41581608
Lasso R2: 0.290199495922
~~~ Ridge ~~~
Ridge MSE:  1672.60490113
Ridge R2: 0.311924817843


### Figuring out the alphas can be done by "hand"

In [23]:
alphas = np.logspace(-10, 10, 21)
for a in alphas:
    print 'Alpha:', a
    lm = linear_model.Ridge(alpha=a)
    lm.fit(modeldata, y)
    print lm.coef_
    print metrics.mean_squared_error(y, lm.predict(modeldata))

Alpha: 1e-10
[ 112.68901765  -84.01121684  -24.68489063  -21.00314493  -21.71893628]
1672.58110765
Alpha: 1e-09
[ 112.68901765  -84.01121684  -24.68489061  -21.00314491  -21.71893626]
1672.58110765
Alpha: 1e-08
[ 112.68901765  -84.01121684  -24.6848904   -21.00314471  -21.71893606]
1672.58110765
Alpha: 1e-07
[ 112.68901763  -84.01121682  -24.68488837  -21.00314268  -21.71893403]
1672.58110765
Alpha: 1e-06
[ 112.68901745  -84.01121667  -24.68486804  -21.00312237  -21.71891373]
1672.58110765
Alpha: 1e-05
[ 112.68901562  -84.01121509  -24.68466472  -21.00291929  -21.71871079]
1672.58110765
Alpha: 0.0001
[ 112.68899732  -84.01119938  -24.68263174  -21.00088873  -21.71668162]
1672.58110765
Alpha: 0.001
[ 112.68881437  -84.01104228  -24.66232204  -20.98060316  -21.69640993]
1672.58110774
Alpha: 0.01
[ 112.68698753  -84.00947323  -24.46121539  -20.77973778  -21.49568404]
1672.58111645
Alpha: 0.1
[ 112.66896732  -83.99396383  -22.63109556  -18.95202277  -19.66942371]
1672.58185208
Alpha: 1.0
[

### Or we can use grid search to make this faster

In [24]:
from sklearn import grid_search

alphas = np.logspace(-10, 10, 21)
gs = grid_search.GridSearchCV(
    estimator=linear_model.Ridge(),
    param_grid={'alpha': alphas},
    scoring='neg_mean_squared_error')

gs.fit(modeldata, y)


/Users/alexj/anaconda3/envs/py27/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


GridSearchCV(cv=None, error_score='raise',
       estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'alpha': array([  1.00000e-10,   1.00000e-09,   1.00000e-08,   1.00000e-07,
         1.00000e-06,   1.00000e-05,   1.00000e-04,   1.00000e-03,
         1.00000e-02,   1.00000e-01,   1.00000e+00,   1.00000e+01,
         1.00000e+02,   1.00000e+03,   1.00000e+04,   1.00000e+05,
         1.00000e+06,   1.00000e+07,   1.00000e+08,   1.00000e+09,
         1.00000e+10])},
       pre_dispatch='2*n_jobs', refit=True,
       scoring='neg_mean_squared_error', verbose=0)

##### Best score 

In [25]:
gs.best_score_ 

-1814.0936913337962

##### mean squared error here comes in negative, so let's make it positive.

In [17]:
-gs.best_score_ 

1814.0936913337962

##### explains which grid_search setup worked best

In [26]:
gs.best_estimator_ 

Ridge(alpha=10.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

##### shows all the grid pairings and their performances.

In [27]:
for s in gs.grid_scores_:
    print s

mean: -1817.58711, std: 542.14315, params: {'alpha': 1e-10}
mean: -1817.58711, std: 542.14315, params: {'alpha': 1.0000000000000001e-09}
mean: -1817.58711, std: 542.14315, params: {'alpha': 1e-08}
mean: -1817.58711, std: 542.14315, params: {'alpha': 9.9999999999999995e-08}
mean: -1817.58711, std: 542.14315, params: {'alpha': 9.9999999999999995e-07}
mean: -1817.58711, std: 542.14317, params: {'alpha': 1.0000000000000001e-05}
mean: -1817.58707, std: 542.14331, params: {'alpha': 0.0001}
mean: -1817.58663, std: 542.14477, params: {'alpha': 0.001}
mean: -1817.58230, std: 542.15933, params: {'alpha': 0.01}
mean: -1817.54318, std: 542.30102, params: {'alpha': 0.10000000000000001}
mean: -1817.20111, std: 543.63587, params: {'alpha': 1.0}
mean: -1814.09369, std: 556.35563, params: {'alpha': 10.0}
mean: -1818.51694, std: 653.68607, params: {'alpha': 100.0}
mean: -2125.58777, std: 872.45270, params: {'alpha': 1000.0}
mean: -2458.08836, std: 951.30428, params: {'alpha': 10000.0}
mean: -2532.21151,

### ACTIVITY

## Gradient Descent

In [30]:
num_to_approach, start, steps, optimized = 6.2, 0., [-1, 1], False
while not optimized:
    current_distance = num_to_approach - start
    got_better = False
    next_steps = [start + i for i in steps]
    for n in next_steps:
        distance = np.abs(num_to_approach - n)
        if distance < current_distance:
            got_better = True
            print distance, 'is better than', current_distance
            current_distance = distance
            start = n
    if got_better:
        print 'found better solution! using', current_distance
    else:
        optimized = True
        print start, 'is closest to', num_to_approach


5.2 is better than 6.2
found better solution! using 5.2
4.2 is better than 5.2
found better solution! using 4.2
3.2 is better than 4.2
found better solution! using 3.2
2.2 is better than 3.2
found better solution! using 2.2
1.2 is better than 2.2
found better solution! using 1.2
0.2 is better than 1.2
found better solution! using 0.2
6.0 is closest to 6.2


### Bonus: 
implement a stopping point, similar to what n_iter would do in gradient descent when we've reached "good enough"

## Demo: Application of Gradient Descent 

In [31]:
lm = linear_model.SGDRegressor()
lm.fit(modeldata, y)
print "Gradient Descent R2:", lm.score(modeldata, y)
print "Gradient Descent MSE:", metrics.mean_squared_error(y, lm.predict(modeldata))

Gradient Descent R2: 0.30792039246
Gradient Descent MSE: 1682.33904312


### Check: Untuned, how well did gradient descent perform compared to OLS?

Answer: 

# Independent Practice: Bike data revisited

There are tons of ways to approach a regression problem. The regularization techniques appended to ordinary least squares optimizes the size of coefficients to best account for error. Gradient Descent also introduces learning rate (how aggressively do we solve the problem), epsilon (at what point do we say the error margin is acceptable), and iterations (when should we stop no matter what?)

For this deliverable, our goals are to:

- implement the gradient descent approach to our bike-share modeling problem,
- show how gradient descent solves and optimizes the solution,
- demonstrate the grid_search module!

While exploring the Gradient Descent regressor object, you'll build a grid search using the stochastic gradient descent estimator for the bike-share data set. Continue with either the model you evaluated last class or the simpler one from today. In particular, be sure to implement the "param_grid" in the grid search to get answers for the following questions:

- With a set of alpha values between 10^-10 and 10^-1, how does the mean squared error change?
- Based on the data, we know when to properly use l1 vs l2 regularization. By using a grid search with l1_ratios between 0 and 1 (increasing every 0.05), does that statement hold true? If not, did gradient descent have enough iterations?
- How do these results change when you alter the learning rate (eta0)?

**Bonus**: Can you see the advantages and disadvantages of using gradient descent after finishing this exercise?

### Starter Code

In [ ]:
params = {} # put your gradient descent parameters here
gs = grid_search.GridSearchCV(
    estimator=linear_model.SGDRegressor(),
    cv=cross_validation.KFold(len(modeldata), n_folds=5, shuffle=True),
    param_grid=params,
    scoring='mean_squared_error',
    )

gs.fit(modeldata, y)

print 'BEST ESTIMATOR'
print -gs.best_score_
print gs.best_estimator_
print 'ALL ESTIMATORS'
print gs.grid_scores_

In [19]:
## go for it!